# Mixup to improve training results


In [1]:
%%capture
# !pip install smdebug torch torchvision tqdm

In [2]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

In [134]:
test

[('2', 'data/metadata/03118.json'),
 ('2', 'data/metadata/02563.json'),
 ('3', 'data/metadata/07574.json'),
 ('2', 'data/metadata/05159.json'),
 ('5', 'data/metadata/04176.json'),
 ('3', 'data/metadata/100306.json'),
 ('1', 'data/metadata/04850.json'),
 ('2', 'data/metadata/102034.json'),
 ('2', 'data/metadata/03414.json'),
 ('4', 'data/metadata/08896.json'),
 ('4', 'data/metadata/103650.json'),
 ('5', 'data/metadata/104545.json'),
 ('5', 'data/metadata/03960.json'),
 ('3', 'data/metadata/04246.json'),
 ('4', 'data/metadata/01290.json'),
 ('1', 'data/metadata/06008.json'),
 ('5', 'data/metadata/09649.json'),
 ('2', 'data/metadata/103003.json'),
 ('2', 'data/metadata/10407.json'),
 ('2', 'data/metadata/103061.json'),
 ('2', 'data/metadata/08118.json'),
 ('3', 'data/metadata/08481.json'),
 ('1', 'data/metadata/07763.json'),
 ('5', 'data/metadata/102112.json'),
 ('3', 'data/metadata/03865.json'),
 ('4', 'data/metadata/09640.json'),
 ('3', 'data/metadata/09516.json'),
 ('2', 'data/metadata

In [13]:
import json
import random

def random_split(filename: str):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # load json
    res = json.loads(lines[0]) 

    # split file to train, test
    l = [(k,bin) for k,v in res.items() for bin in v]

    random.shuffle(l)
    train_size = int(len(l)*0.8)
    train, test = l[: train_size], l[train_size:]
    return train, test

In [145]:
# save file to json folder and maintain struture of original file

def save_json(file):
    file_json = {
    "1": [],
    "2": [],
    "3": [],
    "4": [],
    "5": []
    }

    if file == train_list:
        filename = 'train_list.json'
    else:
        filename = 'test_list.json'
    # filename = str(file) + ".json"  

    for k, i in file:
        file_json[k].append(i)
    
    # sum([len(x) for x in test_json.values()])
    with open(filename, "w") as fp:
        json.dump(file_json, fp)
    


In [5]:
train_list, test_list = random_split("file_list.json")


In [147]:
save_json(train_list)
save_json(test_list)

In [41]:
# with open('train_list.json') as f:
#     train_json = json.load(f)

# train_plot = {
#     "col_names": [],
#     "total_items":[]
# }

# for key, value in train_json.items():
#     train_plot["col_names"].append(key)
#     train_plot["total_items"].append(len(value))

# train_plot

{'col_names': ['1', '2', '3', '4', '5'],
 'total_items': [1011, 1852, 2107, 1891, 1491]}

In [1]:
# df = pd.DataFrame(data=train_plot)
# df.head()

In [2]:
# df.plot(kind ='bar', x="col_names", y="total_items", color='red')

In [3]:
# with open('test_list.json') as f:
#     json_data = json.load(f)

# print(type(json_data))
# print(json_data)


In [4]:
# test_plot = {
#     "col_names": [],
#     "col_len": []
# } 

# for k, v in json_data.items():
#     test_plot["col_names"].append(k)
#     test_plot["col_len"].append(len(v))

# test_plot

In [5]:
# import pandas as pd
# import matplotlib.pyplot as plt

# df = pd.DataFrame(test_plot)
# df

In [7]:
# df.plot(kind ='bar', x="col_names", y="col_len", color='green')
# 


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader

import copy
import argparse
import os
import logging
import sys
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#rom torch_snippets import Report
#from torch_snippets import *

logger=logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

torch.manual_seed(17)


In [4]:
def create_data_loaders(train_dir, test_dir, batch_size, test_batch_size, num_cpus, num_gpus):

    # data augmentation and normalization for train data

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop((224,224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness =0.5), #applies random brightness
        transforms.RandomRotation(degrees =45),
        transforms.RandomVerticalFlip(p=0.05),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # normalization but no data augmentation for test data
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    worker_count = 4
    worker_count =4
    if num_gpus > 0:
        worker_count = min(num_gpus, worker_count)
    elif num_cpus > 0:
        worker_count = min(num_cpus, worker_count)

    train_dataset = ImageFolder(train_dir, transform=train_transform)
    test_dataset = ImageFolder(test_dir, transform=train_transform)
#     test_dataset = ImageFolder(os.path.join(data_dir, 'test'), transform=test_transform)

    train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=worker_count, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size, num_workers=worker_count, pin_memory=True, shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size, num_workers=worker_count)

    return train_loader, test_loader 

In [5]:
def mixup_data(x, y, alpha):

    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index,:]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


In [6]:
# os.listdir()

In [4]:
train_loader, test_loader = create_data_loaders('train_data', 'test_data', 32, 64, 4, 1)

In [66]:
len(train_loader)

261

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def net():
    '''
    TODO: Complete this function that initializes your model
          Remember to use a pretrained model
    '''
    # load the pretrained model
    model = models.resnet101(pretrained=True)
    
#     freeze model weights
    # for param in model.parameters():
    #     param.requires_grad = False
    for param in list(model.parameters())[:-10]:
        param.requires_grad = False
        
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 5)

    return model

In [19]:
n = models.resnet50(pretrained = True)

# for param in n.parameters():
#     param.requires_grad = False
for name, params in list(n.named_parameters())[:-10]:
    params.requires_grad = False
    print(f"{name}:{params.requires_grad}")

for name, params in n.named_parameters():
    print(f"{name}: {params.requires_grad}")

conv1.weight:False
bn1.weight:False
bn1.bias:False
layer1.0.conv1.weight:False
layer1.0.bn1.weight:False
layer1.0.bn1.bias:False
layer1.0.conv2.weight:False
layer1.0.bn2.weight:False
layer1.0.bn2.bias:False
layer1.0.conv3.weight:False
layer1.0.bn3.weight:False
layer1.0.bn3.bias:False
layer1.0.downsample.0.weight:False
layer1.0.downsample.1.weight:False
layer1.0.downsample.1.bias:False
layer1.1.conv1.weight:False
layer1.1.bn1.weight:False
layer1.1.bn1.bias:False
layer1.1.conv2.weight:False
layer1.1.bn2.weight:False
layer1.1.bn2.bias:False
layer1.1.conv3.weight:False
layer1.1.bn3.weight:False
layer1.1.bn3.bias:False
layer1.2.conv1.weight:False
layer1.2.bn1.weight:False
layer1.2.bn1.bias:False
layer1.2.conv2.weight:False
layer1.2.bn2.weight:False
layer1.2.bn2.bias:False
layer1.2.conv3.weight:False
layer1.2.bn3.weight:False
layer1.2.bn3.bias:False
layer2.0.conv1.weight:False
layer2.0.bn1.weight:False
layer2.0.bn1.bias:False
layer2.0.conv2.weight:False
layer2.0.bn2.weight:False
layer2.0.bn2

In [20]:
# !pip install torchsummary

m = net().to(device)
# for name, param in m.named_parameters():
#     print(f"{name}: {param.shape}")

from torchsummary import summary

summary(m, (3, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to C:\Users\rapha/.cache\torch\hub\checkpoints\resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:25<00:00, 7.06MB/s] 


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [8]:
# %%capture
# pip install torchmetrics
# pip3 install torch torchvision torchaudio
import torchmetrics
from torchmetrics.functional import accuracy
print(torchmetrics.__version__)

0.2.0


In [9]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0
    running_corrects = 0
    alpha = 0.9
    num_classes = 5
    acc = torchmetrics.Accuracy()
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs,y1,y2, lam = mixup_data(inputs, labels, alpha)
            labels = lam *  y1 + (1 - lam) * y2
            outputs=model(inputs)
            loss=criterion(outputs, labels.long())
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            batch_acc = acc(preds.cpu(), labels.long().cpu())

    total_loss = running_loss // len(test_loader)
    total_acc = running_corrects.double() // len(test_loader)
    new_acc = acc.compute()

    acc.reset()
    
    print(f"Testing Loss: {total_loss}")
    print(f"Testing Accuracy: {total_acc} | new_acc: {new_acc}")

In [10]:
def train(model, train_loader, criterion, optimizer):
    model.train() # set model to training mode
    running_loss = 0
    correct = 0
    alpha = 0.8
    num_classes = 5
    train_accuracy = torchmetrics.Accuracy().to(device)

    # iterate over the data
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs,y1,y2, lam = mixup_data(inputs, labels, alpha)
        labels = lam *  y1 + (1 - lam) * y2
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(inputs)
#       preds = outputs.argmax(dim=1, keepdim=True)
        _, preds = torch.max(outputs, dim=1)
#       correct += preds.eq(target.view_as(preds)).sum().item()
        correct += torch.sum(preds == labels).item()
        loss = criterion(outputs, labels.long())
        running_loss += loss.item() * inputs.size(0)
#       logger.info(f"loss: {loss}| acc: {torch.sum(preds==target).item()/len(target)}")
        acc = train_accuracy(preds, labels.long())
        
        # backward + optimize
        loss.backward()
        optimizer.step()
    train_loss = loss/len(train_loader.dataset)
    train_acc = correct / len(train_loader.dataset)
    new_acc = train_accuracy.compute()

    train_accuracy.reset()

    logger.info(f"\nTrain set: Average loss: {train_loss:.4f}| Accuracy: {100.0 * train_acc:.4f}% | new_acc: {new_acc}")

In [11]:
batch_size =32
epochs = 10
learning_rate= 1e-4 #0.0003667366574731279
train_loader, test_loader = create_data_loaders('train_data', 'test_data', 32, 32, 4, 1)
model=net()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

print("Starting Model Training")
for epoch in range(1, epochs + 1):
        print(f"\nEpoch: {epoch}\nTraining")
        train(model, train_loader, criterion, optimizer)
# #         logger.info(f"validating...")
# #         test(model, valid_loader, loss_criterion, device)
        
    
'''
TODO: Test the model to see its accuracy
'''
print(f"Testing ...")
test(model, test_loader, criterion)

Starting Model Training

Epoch: 1
Training

Train set: Average loss: 0.0002| Accuracy: 7.0163% | new_acc: 0.32519155740737915

Epoch: 2
Training

Train set: Average loss: 0.0002| Accuracy: 6.8367% | new_acc: 0.3316570818424225

Epoch: 3
Training

Train set: Average loss: 0.0002| Accuracy: 7.4713% | new_acc: 0.3475814163684845

Epoch: 4
Training

Train set: Average loss: 0.0002| Accuracy: 7.3875% | new_acc: 0.3516522943973541

Epoch: 5
Training

Train set: Average loss: 0.0002| Accuracy: 7.1839% | new_acc: 0.3585967421531677

Epoch: 6
Training

Train set: Average loss: 0.0002| Accuracy: 7.1959% | new_acc: 0.35955458879470825

Epoch: 7
Training

Train set: Average loss: 0.0002| Accuracy: 7.6149% | new_acc: 0.3563218414783478

Epoch: 8
Training

Train set: Average loss: 0.0002| Accuracy: 7.4832% | new_acc: 0.3506944477558136

Epoch: 9
Training

Train set: Average loss: 0.0002| Accuracy: 6.7050% | new_acc: 0.3500957787036896

Epoch: 10
Training

Train set: Average loss: 0.0001| Accuracy: 7

In [61]:
# plot data distribution